# Counting Usage of Lemmas 



This script attempts to evaluate the project 

The first stage is finding the usage of lemmas towards the declaration

A further step is to score the difficulty of lemmas given their counts of the basic lemmas
    refl
    sym
    trans
    cong 
    ε-right
    ε-left 
    ∙-assoc
    

In [257]:
from __future__ import unicode_literals
import re

import pandas as pd
import numpy as np

In [258]:
agda_file_location = 'agda-playground/fingertree/FingerTree-measure-size-c.agda'
#agda_file_location = 'agda-playground/fingertree/list-reverse-proof.agda'

#requirements are that all declarations are separated by at least a newline

#as a styling techinique to make this easier:
# all lemmas contain the word lemma
# lemmas that are derived through the monoid solver have a dash (') at the end


In [277]:
# regular expressions

#comment in the begining
comment_begining = r'^\s*--'

#imports 
imports_begining = r'^(\s*open)|(\s*open\s*import)|(\s*import)'

#infixation
infix_begining = r'^\s*infix[lr]*'

#instance
instance_begining = r'^\s*instance'

#data
data_begining = r'^\s*data'

#mutual
mutual_begining = r'^\s*mutual'

#module 
module = r'^\s*module'

#declarations
decl = r'^\s*(postulate){0,1}\s*([^\s:]+)\s*:'

#lemmas 
lemmas = [r'refl',
          r'sym',
          r'cong',
          r'≡⟨',          
          r'ε-left',
          r'ε-right',
          r'∙-assoc',
          r'[^\s:\(]+lemma[^\s:\)]*',    #two ways through which
          r'[^\s:\(]+correct[^\s:\)]*']  #I identify lemmas in the src.

#lemma building blocks.
basic_lemmas = [r'refl',
          r'sym',
          r'cong',
          r'≡⟨',     #this is equivalent to a trans application
          r'ε-left',
          r'ε-right',
          r'∙-assoc']


#for pretty printing
basic_lemmas_names = [r'refl',
          r'sym',
          r'cong',
          r'trans',    
          r'ε-left',
          r'ε-right',
          r'∙-assoc']                

In [278]:
import codecs

agda_file = codecs.open(agda_file_location, 'r', encoding='utf-8')
lines = agda_file.readlines()

In [279]:
def remove_all_lines(reg, lines):
    filter_fun = lambda x : (re.match(reg, x) == None)
    return [x for x in lines if filter_fun(x)]

def remove_ending_comment(line):
    return line.split('--')[0]

def strip_ending_comments(lines):
    return [remove_ending_comment(x) for x in lines]


In [280]:
#preprocessing the lines -- decluttering the document 


lines = remove_all_lines(comment_begining, lines)
lines = remove_all_lines(imports_begining, lines)
lines = remove_all_lines(infix_begining, lines)
lines = remove_all_lines(mutual_begining, lines)
lines = remove_all_lines(module, lines)
lines = strip_ending_comments(lines)

In [281]:
# at least * all top level declarations 
#these are toplogically sorted

declarations = [re.search(decl, x).group(1) for x in lines if re.search(decl,x) is not None]

In [282]:
#splitting in sections 

new_lines = [i for (i, x) in enumerate(lines) if x == u'\n']

#partition to sections
#that is, each section corresponds to a declaration
def partition(new_lines, lines):
    parts = []
    N = len (new_lines)
    for i in range(1, N):
        parts.append(lines[new_lines[i-1] + 1 : new_lines[i]])
    return parts

parts = partition(new_lines, lines)

#remove empty lists 
parts = [x for x in parts if len(x) > 0]

#select only sections that are not data declarations
#we are only interested in function declarations
def select_parts_begining(regex, parts):
    filter_fun = lambda x : (re.match(regex, x[0]) == None) 
    return [x for x in parts if filter_fun(x)]

decl_parts = select_parts_begining(data_begining, parts)

#select all sections that are not instance declarations -- makes things easier
decl_parts = select_parts_begining(instance_begining, decl_parts)

#identify declarations in each part
def parts_dict(parts):
    p_dict = {}
    for part in parts:
        line = part[0]
        decl_match = re.search(decl, line)
        if (decl_match is None):
            print part
        declr = decl_match.group(2)
        assert declr not in p_dict
        p_dict[declr] = part
    return p_dict

# find all the lemmas used in a declaration(part)
def find_lemmas(decl, part):
    #outputs {n_lines, rec, lemmas}
    results = []    
    n_lines = len(part)
    rec = False 
    
    for regex in lemmas:
        for line in part:
            results += re.findall(regex, line)
        #    print results
    results = [x for x in results if not (x == decl)]
    
    #find whether it is recursive -- TODO    
    rec_regex = r'\s+[^\s]+\s+' + decl
    # don't want the declaration to appear at new line, 
    # because that's just caused by initialisation    
    rec_list = []
    for line in part: 
        rec_list += re.findall(rec_regex, line)        
    if rec_list is not []:
        rec = True;
        
    return {'n_lines' : n_lines, 'rec' : rec, 'lemmas' : results}
    

In [283]:
decl_dict = parts_dict(decl_parts)

In [284]:
def lemma_dict(decl_dict):
    result = {}
    for key in decl_dict:
        result[key] = find_lemmas(key, decl_dict[key])['lemmas']
    return result


# other lemmas stats

def declaration_stats(decl_dict):
    result = {}
    for key in decl_dict:
        lemma_stat = find_lemmas(key, decl_dict[key])
        result[key] = lemma_stat['n_lines'], lemma_stat['rec']
    result = pd.DataFrame.from_dict(result, 'index')
    result.columns = ['n_lines', 'rec']
    return result

In [285]:
lemma_usage = lemma_dict(decl_dict)
lemma_usage['foldl-correct']

[u'refl',
 u'refl',
 u'refl',
 u'sym',
 u'sym',
 u'cong',
 u'cong',
 u'cong',
 u'\u2261\u27e8',
 u'\u2261\u27e8',
 u'\u2261\u27e8',
 u'\u2261\u27e8',
 u'\u2261\u27e8',
 u'\u2261\u27e8',
 u'flatten-fold-lemma',
 u'foldl-append-lemma2',
 u'foldl-dig-correct',
 u'foldl-dig-correct']

In [286]:
#this fills up automatically because of memoisation
flattened_lemmas = {}

def flatten_list(l):
    result = []
    if type(element) is str:
        result.append(element)
    else:
        result += element
    return result

def flatten_lemma(decl, lemma_dict):
    if decl in flattened_lemmas:
        return flattened_lemmas[decl]
    
    if decl in lemma_dict:
        used_lemmas = lemma_dict[decl]
    else:
        used_lemmas = []
        
    for (i, lemma) in enumerate(used_lemmas):
        if lemma not in basic_lemmas:
            flat = flatten_lemma(lemma, lemma_dict)
            used_lemmas[i:i+1] = flat
             
    flattened_lemmas[decl] = used_lemmas
    return used_lemmas    

In [287]:
def flatten_dict(lemma_usage):
    for key in lemma_usage:
        flatten_lemma(key, lemma_usage)

flatten_dict(lemma_usage)

lemma_dict = flattened_lemmas


# normalizes a lemma list as a frequency vector of basic lemmas usage
def normalize_lemma_vec(lemmas):
    result = np.zeros(len(basic_lemmas))
    for lemma in lemmas:
        if lemma in basic_lemmas:
            i = basic_lemmas.index(lemma)
            result[i] += 1
    return result

# normalize the whole dictionary
def normalize_lemma_dict(lemma_dict):
    result = {}
    for key in lemma_dict:
        result[key] = normalize_lemma_vec(lemma_dict[key])
    return result

# alternative lemma dictionary with unique lemmas
def all_lemma_dict(lemma_dict):
    result = {}
    for key in lemma_dict:
        result[key] = set(lemma_dict[key])
    return result

norm_lemma_dict = normalize_lemma_dict(lemma_dict)

In [300]:
#need to find a way to analyze this data
df_lemmas = pd.DataFrame.from_dict(norm_lemma_dict, orient='index', dtype=int)
df_lemmas.columns = basic_lemmas_names
df_lemmas.loc['deepL'] + df_lemmas.loc['viewL']

refl       3
sym        5
cong       1
trans      6
ε-left     4
ε-right    2
∙-assoc    3
dtype: int64

In [289]:
all_lemma_dict(lemma_usage)

{u'FunExt': set(),
 u'_\u25b7_': {u'cong',
  u'sym',
  u'\u03b5-left',
  u'\u2219-assoc',
  u'\u2261\u27e8'},
 u'_\u25c1_': {u'cong',
  u'refl',
  u'sym',
  u'\u03b5-left',
  u'\u03b5-right',
  u'\u2219-assoc',
  u'\u2261\u27e8'},
 u'append': {u'cong',
  u'refl',
  u'\u03b5-right',
  u'\u2219-assoc',
  u'\u2261\u27e8'},
 u'append-measure-lemma': {u'cong', u'refl', u'\u2219-assoc', u'\u2261\u27e8'},
 u'assoc-lemma1': {u'cong', u'sym', u'\u03b5-left'},
 u'assoc-lemma2': {u'cong', u'refl', u'sym', u'\u2219-assoc', u'\u2261\u27e8'},
 u'assoc-lemma3': {u'sym', u'\u03b5-left', u'\u2219-assoc', u'\u2261\u27e8'},
 u'assoc-lemma4': {u'sym', u'\u03b5-left', u'\u2219-assoc', u'\u2261\u27e8'},
 u'assoc-lemma5': {u"assoc-lemma5'"},
 u'assoc-lemma6': {u"assoc-lemma6'"},
 u'assoc-lemma7': {u"assoc-lemma7'"},
 u'assoc-lemma8': {u"assoc-lemma8'"},
 u'assoc-lemma9': {u"assoc-lemma9'"},
 u'concat': {u'cong',
  u'refl',
  u'sym',
  u'\u03b5-left',
  u'\u03b5-right',
  u'\u2219-assoc',
  u'\u2261\u27e8'},


≡⟨
